In [1]:
import numpy as np
import pandas as pd
from math import inf
import torch
import torchmetrics
from torch.nn import CrossEntropyLoss, NLLLoss
import torch.nn as nn

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

from spotPython.spot import spot
from spotPython.utils.init import fun_control_init
from spotPython.hyperparameters.values import (
    add_core_model_to_fun_control,
    modify_hyper_parameter_levels,
    modify_hyper_parameter_bounds,
    get_var_type,
    get_var_name,
    get_bound_values,
    get_one_core_model_from_X,
    get_default_hyperparameters_as_array
    )
from spotPython.data.torch_hyper_dict import TorchHyperDict
from spotPython.fun.hypertorch import HyperTorch
from spotPython.torch.netvbdp import Net_vbdp
from spotPython.torch.traintest import (
    train_tuned,
    test_tuned,
    )
from spotPython.torch.dataframedataset import DataFrameDataset
from spotPython.torch.mapk import MAPK
from spotPython.data.vbdp import modify_vbdp_dataframe, combine_features

# Feature Engineering

In [2]:
fun_control = fun_control_init(task="classification", tensorboard_path="runs/25_spot_torch_vbdp")
fun_control.update({"show_batch_interval": 100_000_000})
# load data
train_df = pd.read_csv('./data/VBDP/train.csv')
# remove the id column
train_df = train_df.drop(columns=['id'])
n_samples = train_df.shape[0]
n_features = train_df.shape[1] - 1
target_column = "prognosis"
# # Encoder our prognosis labels as integers for easier decoding later
enc = OrdinalEncoder()
train_df[target_column] = enc.fit_transform(train_df[[target_column]])
train_df.head()

# convert all entries to int for faster processing
train_df = train_df.astype(int)

In [3]:
df_new = train_df.copy()
# save the target column using "target_column" as the column name
target = train_df[target_column]
# remove the target column
df_new = df_new.drop(columns=[target_column])
train_df = combine_features(df_new)
# add the target column back
train_df[target_column] = target
train_df.head()

/var/folders/dw/pvtj6mt91znd0hftcztqb0k00000gn/T/ipykernel_12542/3336454340.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[target_column] = target


,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,hypotension,...,6039,6040,6041,6042,6043,6044,6045,6046,6047,prognosis
0,1,1,0,1,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,3
1,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,7
2,0,1,1,1,0,1,1,1,1,1,...,1,1,0,1,1,0,1,1,0,3
3,0,0,1,1,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,10
4,0,0,0,0,0,0,0,0,1,0,...,0,1,1,0,1,1,0,0,0,6


* feature engineering: 6112 features

In [4]:
n_samples = train_df.shape[0]
n_features = train_df.shape[1] - 1
train_df.columns = [f"x{i}" for i in range(1, n_features+1)] + [target_column]
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(target_column, axis=1), train_df[target_column],
                                                    random_state=42,
                                                    test_size=0.25,
                                                    stratify=train_df[target_column])
trainset = pd.DataFrame(np.hstack((X_train, np.array(y_train).reshape(-1, 1))))
testset = pd.DataFrame(np.hstack((X_test, np.array(y_test).reshape(-1, 1))))
trainset.columns = [f"x{i}" for i in range(1, n_features+1)] + [target_column]
testset.columns = [f"x{i}" for i in range(1, n_features+1)] + [target_column]
print(train_df.shape)
print(trainset.shape)
print(testset.shape)


(707, 6113)
(530, 6113)
(177, 6113)


In [5]:
dtype_x = torch.float32
dtype_y = torch.long
train_df = DataFrameDataset(train_df, target_column=target_column, dtype_x=dtype_x, dtype_y=dtype_y)
train = DataFrameDataset(trainset, target_column=target_column, dtype_x=dtype_x, dtype_y=dtype_y)
test = DataFrameDataset(testset, target_column=target_column, dtype_x=dtype_x, dtype_y=dtype_y)
n_samples = len(train)

In [6]:
# add the dataset to the fun_control
fun_control.update({"data": train_df, # full dataset,
               "train": train,
               "test": test,
               "n_samples": n_samples,
               "target_column": target_column})


In [7]:

# add the nn model to the fun_control dictionary
fun_control = add_core_model_to_fun_control(core_model=Net_vbdp,
                              fun_control=fun_control,
                              hyper_dict=TorchHyperDict)
# modify the hyperparameter levels
fun_control = modify_hyper_parameter_bounds(fun_control, "_L0", bounds=[n_features, n_features])
# fun_control = modify_hyper_parameter_bounds(fun_control, "l1", bounds=[3, 4])
# fun_control = modify_hyper_parameter_bounds(fun_control, "epochs", bounds=[2, 9])
fun_control = modify_hyper_parameter_bounds(fun_control, "patience", bounds=[2, 6])
fun_control = modify_hyper_parameter_bounds(fun_control, "lr_mult", bounds=[1e-3, 1e-3])
fun_control = modify_hyper_parameter_bounds(fun_control, "sgd_momentum", bounds=[0.9, 0.9])
fun_control = modify_hyper_parameter_levels(fun_control, "optimizer",["Adam", "AdamW", "Adamax", "NAdam"])
# select metric and loss function
# metric_torch = torchmetrics.Accuracy(task="multiclass", num_classes=11)
metric_torch = MAPK(k=3)
loss_torch = CrossEntropyLoss()
# loss_torch = NLLLoss()
fun_control.update({
               "metric_torch": metric_torch,
               "loss_function": loss_torch,
               "device": "cpu",
               })
# extract the variable types, names, and bounds
var_type = get_var_type(fun_control)
var_name = get_var_name(fun_control)
fun_control.update({"var_type": var_type,
                    "var_name": var_name})
lower = get_bound_values(fun_control, "lower")
upper = get_bound_values(fun_control, "upper")

# get the default hyperparameters as array
hyper_dict=TorchHyperDict().load()
X_start = get_default_hyperparameters_as_array(fun_control, hyper_dict)

# get the objective function
fun = HyperTorch().fun_torch

# initialize spot
spot_tuner = spot.Spot(fun=fun,
                   lower = lower,
                   upper = upper,
                   fun_evals = inf,
                   fun_repeats = 1,
                   max_time = 1,
                   noise = False,
                   tolerance_x = np.sqrt(np.spacing(1)),
                   var_type = var_type,
                   var_name = var_name,
                   infill_criterion = "y",
                   n_points = 1,
                   seed=123,
                   log_level = 50,
                   show_models= False,
                   show_progress= True,
                   fun_control = fun_control,
                   design_control={"init_size": 5,
                                   "repeats": 1},
                   surrogate_control={"noise": True,
                                      "cod_type": "norm",
                                      "min_theta": -4,
                                      "max_theta": 3,
                                      "n_theta": len(var_name),
                                      "model_fun_evals": 10_000,
                                      "log_level": 50
                                      })
# run spot
spot_tuner.run(X_start=X_start)



config: {'_L0': 6112, 'l1': 32768, 'dropout_prob': 0.7103122166156, 'lr_mult': 0.001, 'batch_size': 4, 'epochs': 64, 'k_folds': 1, 'patience': 64, 'optimizer': 'AdamW', 'sgd_momentum': 0.9}
Epoch: 1
Loss on hold-out set: 2.397013398836244
Accuracy on hold-out set: 0.14622641509433962
MAPK value on hold-out data: 0.23506289308176098
Epoch: 2
Loss on hold-out set: 2.3962632890017526
Accuracy on hold-out set: 0.12735849056603774
MAPK value on hold-out data: 0.2272012578616352
Epoch: 3
Loss on hold-out set: 2.3953690034038617
Accuracy on hold-out set: 0.14622641509433962
MAPK value on hold-out data: 0.23977987421383645
Epoch: 4
Loss on hold-out set: 2.394227927585818
Accuracy on hold-out set: 0.13679245283018868
MAPK value on hold-out data: 0.2366352201257861
Epoch: 5
Loss on hold-out set: 2.3928299624964877
Accuracy on hold-out set: 0.13679245283018868
MAPK value on hold-out data: 0.23663522012578614
Epoch: 6
Loss on hold-out set: 2.3910861375196926
Accuracy on hold-out set: 0.1367924528

In [ ]:
#| echo: true
#| eval: false
spot_tuner.plot_progress()

In [ ]:
#| echo: true
#| eval: false
from spotPython.utils.eda import gen_design_table
print(gen_design_table(fun_control=fun_control, spot=spot_tuner))

In [ ]:
#| echo: true
#| eval: false
spot_tuner.plot_importance()

In [ ]:
#| echo: true
#| eval: false
X = spot_tuner.to_all_dim(spot_tuner.min_X.reshape(1,-1))
model_spot = get_one_core_model_from_X(X, fun_control)
model_spot

In [ ]:
from spotPython.torch.mapk import MAPK
metric_torch = MAPK(k=3)
fun_control.update({
               "metric_torch": metric_torch,               
               })

train_tuned(net=model_spot, train_dataset=train,
        loss_function=fun_control["loss_function"],
        metric=fun_control["metric_torch"],
        shuffle=True,
        device = "cpu",
        path=None,
        task=fun_control["task"],)

In [ ]:
test_tuned(net=model_spot, test_dataset=test,
            shuffle=False,
            loss_function=fun_control["loss_function"],
            metric=fun_control["metric_torch"],
            device = "cpu",
            task=fun_control["task"],)

## Cross-validated Evaluations

* This is the evaluation that will be used in the comparison (evaluatecv has to be updated before, to get metric vlaues!):

In [ ]:
from spotPython.torch.traintest import evaluate_cv
# modify k-kolds:+
setattr(model_spot, "k_folds",  10)
evaluate_cv(net=model_spot, dataset=fun_control["data"], loss_function=fun_control["loss_function"], metric=fun_control["metric_torch"], task=fun_control["task"], writer=fun_control["writer"], writerId="model_spot_cv", device="cpu")

In [ ]:
#| echo: true
#| eval: false
spot_tuner.plot_important_hyperparameter_contour()

In [ ]:
# close tensorbaoard writer
if fun_control["writer"] is not None:
    fun_control["writer"].close()